# VectorStore

## Setting Up the Environment

In [10]:
import fitz
import numpy as np
import json
import re
from tqdm import tqdm

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [12]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로.

    Returns:
        str: 추출된 전체 텍스트.
    """
    # PDF 파일을 엽니다.
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 텍스트를 저장할 문자열 초기화

    # 각 페이지를 순회하면서 텍스트를 추출합니다.
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 페이지 가져오기
        text = page.get_text("text")  # 해당 페이지에서 텍스트 추출
        all_text += text  # 추출된 텍스트를 누적

    return all_text  # 전체 텍스트 반환

## Chunking the Extracted Text

In [13]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 지정된 수의 문자가 겹치도록 분할합니다.

    Args:
        text (str): 분할할 원본 텍스트.
        n (int): 각 청크의 문자 수.
        overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
        List[str]: 분할된 텍스트 청크 리스트.
    """
    chunks = []  # 청크를 저장할 빈 리스트 초기화
    
    # (n - overlap)만큼 이동하며 텍스트를 분할
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])  # 현재 위치부터 n자까지 슬라이싱하여 추가

    return chunks  # 청크 리스트 반환

## Generating Questions for Text Chunks

In [14]:
def generate_questions(text_chunk, num_questions=5, model="gpt-4o-mini"):
    """
    주어진 텍스트 청크로부터 관련 질문들을 생성합니다.

    Args:
        text_chunk (str): 질문을 생성할 대상 텍스트 청크.
        num_questions (int): 생성할 질문의 개수.
        model (str): 사용할 언어 모델.

    Returns:
        List[str]: 생성된 질문 리스트.
    """
    # AI의 역할을 정의하는 시스템 프롬프트
    system_prompt = (
        "당신은 텍스트로부터 관련 질문을 생성하는 전문가입니다. "
        "제공된 텍스트를 바탕으로 그 내용에만 근거한 간결한 질문들을 생성하세요. "
        "핵심 정보와 개념에 초점을 맞추세요."
    )
    
    # 사용자 프롬프트: 텍스트와 함께 질문 생성 요청
    user_prompt = f"""
    다음 텍스트를 기반으로, 해당 텍스트만으로 답할 수 있는 서로 다른 질문 {num_questions}개를 생성하세요:

    {text_chunk}
    
    응답은 번호가 매겨진 질문 리스트 형식으로만 작성하고, 그 외 부가 설명은 하지 마세요.
    """
    
    # 모델 호출을 통해 질문 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 응답에서 질문 문자열 추출
    questions_text = response.choices[0].message.content.strip()
    questions = []

    # 줄 단위로 질문을 추출하고 정리
    for line in questions_text.split('\n'):
        # 번호 제거 및 양쪽 공백 제거
        cleaned_line = re.sub(r'^\d+\.\s*', '', line.strip())
        if cleaned_line and cleaned_line.endswith('?'):
            questions.append(cleaned_line)

    return questions

## Creating Embeddings for Text

In [15]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    지정된 모델을 사용하여 입력 텍스트에 대한 임베딩을 생성합니다.

    Args:
        text (str): 임베딩을 생성할 입력 텍스트 또는 텍스트 리스트.
        model (str): 사용할 임베딩 모델 이름.

    Returns:
        dict: 생성된 임베딩 정보를 포함한 OpenAI API의 응답 객체.
    """
    # 입력 텍스트에 대해 임베딩 생성 요청
    response = client.embeddings.create(
        model=model,
        input=text
    )

    # 응답 객체 반환
    return response

## Building a Simple Vector Store

In [16]:
class SimpleVectorStore:
    """
    NumPy를 사용한 간단한 벡터 저장소 구현 클래스입니다.
    """
    def __init__(self):
        """
        벡터 저장소를 초기화합니다.
        """
        self.vectors = []    # 임베딩 벡터들을 저장
        self.texts = []      # 원본 텍스트들을 저장
        self.metadata = []   # 텍스트에 대한 메타데이터 저장
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목을 추가합니다.

        Args:
            text (str): 원본 텍스트.
            embedding (List[float]): 임베딩 벡터.
            metadata (dict, optional): 추가 메타데이터 (기본값: None).
        """
        self.vectors.append(np.array(embedding))             # 벡터 추가
        self.texts.append(text)                              # 텍스트 추가
        self.metadata.append(metadata or {})                 # 메타데이터 추가
    
    def similarity_search(self, query_embedding, k=5):
        """
        쿼리 임베딩과 가장 유사한 항목을 검색합니다.

        Args:
            query_embedding (List[float]): 쿼리 벡터.
            k (int): 반환할 결과 수 (기본값: 5).

        Returns:
            List[Dict]: 상위 k개의 유사 항목. 텍스트, 메타데이터, 유사도 포함.
        """
        if not self.vectors:
            return []
        
        query_vector = np.array(query_embedding)
        similarities = []

        # 각 벡터와의 코사인 유사도 계산
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })
        
        return results

## Processing Documents with Question Augmentation

In [17]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200, questions_per_chunk=5):
    """
    문서를 처리하고, 각 청크에 대해 질문을 생성하여 벡터 저장소에 추가합니다.

    Args:
        pdf_path (str): PDF 파일 경로.
        chunk_size (int): 각 청크의 문자 수.
        chunk_overlap (int): 청크 간 중첩 문자 수.
        questions_per_chunk (int): 청크당 생성할 질문 수.

    Returns:
        Tuple[List[str], SimpleVectorStore]: 생성된 텍스트 청크 리스트와 벡터 저장소 객체.
    """
    print("PDF에서 텍스트 추출 중...")
    extracted_text = extract_text_from_pdf(pdf_path)
    
    print("텍스트 청크 분할 중...")
    text_chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"총 {len(text_chunks)}개의 텍스트 청크가 생성되었습니다.")
    
    vector_store = SimpleVectorStore()
    
    print("각 청크에 대해 임베딩 및 질문 생성 중...")
    for i, chunk in enumerate(tqdm(text_chunks, desc="청크 처리 중")):
        # 청크 임베딩 생성
        chunk_embedding_response = create_embeddings(chunk)
        chunk_embedding = chunk_embedding_response.data[0].embedding
        
        # 청크를 벡터 저장소에 추가
        vector_store.add_item(
            text=chunk,
            embedding=chunk_embedding,
            metadata={"type": "chunk", "index": i}
        )
        
        # 해당 청크 기반 질문 생성
        questions = generate_questions(chunk, num_questions=questions_per_chunk)
        
        # 각 질문에 대한 임베딩 생성 후 저장소에 추가
        for j, question in enumerate(questions):
            question_embedding_response = create_embeddings(question)
            question_embedding = question_embedding_response.data[0].embedding
            
            vector_store.add_item(
                text=question,
                embedding=question_embedding,
                metadata={
                    "type": "question",
                    "chunk_index": i,
                    "original_chunk": chunk
                }
            )
    
    return text_chunks, vector_store

## Extracting and Processing the Document

In [18]:
# PDF 파일 경로 정의
pdf_path = "../dataset/AI_Understanding.pdf"

# 문서 처리: 텍스트 추출, 청크 분할, 질문 생성, 벡터 저장소 구축
text_chunks, vector_store = process_document(
    pdf_path, 
    chunk_size=1000,       # 각 청크는 1000자
    chunk_overlap=200,     # 청크 간 200자 중첩
    questions_per_chunk=3  # 청크당 질문 3개 생성
)

# 벡터 저장소에 저장된 항목 개수 출력
print(f"벡터 저장소에 저장된 항목 수: {len(vector_store.texts)}개")

PDF에서 텍스트 추출 중...
텍스트 청크 분할 중...
총 21개의 텍스트 청크가 생성되었습니다.
각 청크에 대해 임베딩 및 질문 생성 중...


청크 처리 중: 100%|██████████| 21/21 [01:22<00:00,  3.91s/it]

벡터 저장소에 저장된 항목 수: 84개
